In [48]:
import torch
from torch import nn
import torchvision
from torchvision import transforms
import matplotlib.pyplot as plt
from collections import namedtuple
from sklearn.metrics import classification_report

In [51]:
## CIFAR-10
classes = ('plane', 'car', 'bird', 'cat',
          'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
TrainTest = namedtuple('TrainTest', ['train', 'test'])

def prepare_data():
    transform_train = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor()
    ])
    transform_test = transforms.Compose([
        transforms.ToTensor()
    ])
    trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
    testset  = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
    return TrainTest(
        train=trainset,
        test=testset
    )

def prepare_loader(datasets):
    batch_size = 128
    trainloader = torch.utils.data.DataLoader(
        datasets.train, batch_size=batch_size, shuffle=True, num_workers=4)
    testloader = torch.utils.data.DataLoader(
        datasets.test, batch_size=batch_size, shuffle=False, num_workers=4)
    return TrainTest(
        train=trainloader,
        test=testloader
    )

class VGG16(nn.Module):
    def __init__(self):
        super().__init__()
        self.features = self._make_layers()
        self.classfier_head = nn.Linear(512, 10)
        
    def forward(self, x):
        out = self.features(x)
        out = self.classfier_head(out.view(out.size(0), -1))
        return out
    
    def _make_layers(self):
        config = [64, 64, 'MP', 128, 128, 'MP', 256, 256, 256, 'MP', 512, 512, 512, 'MP', 512, 512, 512, 'MP']
        layers = []
        c_in = 3
        for c in config:
            if c == 'MP':
                layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
            else:
                layers += [
                    nn.Conv2d(in_channels=c_in, out_channels=c, kernel_size=3, padding=1),
                    nn.BatchNorm2d(c),
                    nn.ReLU6(inplace=True)
                ]
                c_in = c
        return nn.Sequential(*layers)
    
def get_trainer(model):
    loss = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)
    return loss, optimizer

def train_epoch(epoch, model, loader, loss_func, optimizer, device):
    model.train()
    running_loss = 0.0
    reporting_step = 60
    for i, (images, labels) in enumerate(loader):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = loss_func(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        if i % reporting_step == reporting_step-1:
            print(f"Epoch {epoch} Step {i} ave_loss {running_loss/reporting_step:0.4f}")
            running_loss = 0.0
            
def test_epoch(epoch, model, loader, device):
    model.eval()
    ypred = []
    ytrue = []
    for i, (images, labels) in enumerate(loader):
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs, dim=1)
        ypred += list(predicted.cpu().numpy())
        ytrue += list(labels.cpu().numpy())
    return ypred, ytrue
        

In [52]:
def main(PATH='./model.pth'):
    datasets = prepare_data()
    loaders = prepare_loader(datasets)
    model = VGG16()
    loss, optimizer = get_trainer(model)
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    
    n_epoch = 10   
    model.to(device)
    for epoch in range(n_epoch):
        train_epoch(epoch, model, loaders.train, loss, optimizer, device)
        ypred, ytrue = test_epoch(epoch, model, loaders.test, device)
        print(classification_report(ytrue, ypred, target_names=classes))
        
        torch.save(model.state_dict(), PATH)
        
    return model

model = main()

Files already downloaded and verified
Files already downloaded and verified
data 50000 10000
in torch.Size([128, 3, 32, 32]) torch.Size([128])
out torch.Size([128, 10])
cuda:0
Epoch 0 Step 59 ave_loss 2.0802
Epoch 0 Step 119 ave_loss 1.7307
Epoch 0 Step 179 ave_loss 1.5541
Epoch 0 Step 239 ave_loss 1.3917
Epoch 0 Step 299 ave_loss 1.2736
Epoch 0 Step 359 ave_loss 1.2007
              precision    recall  f1-score   support

       plane       0.46      0.83      0.59      1000
         car       0.83      0.70      0.76      1000
        bird       0.34      0.50      0.40      1000
         cat       0.34      0.59      0.43      1000
        deer       0.57      0.25      0.34      1000
         dog       0.75      0.20      0.32      1000
        frog       0.72      0.67      0.69      1000
       horse       0.78      0.48      0.60      1000
        ship       0.69      0.82      0.75      1000
       truck       0.83      0.61      0.71      1000

    accuracy                   